# Import

In [1]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_mutual_info_score

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Read data

In [3]:
ag_news_df = pd.read_csv("../../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [4]:
with open("../../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [5]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [6]:
label = ag_news_df["class"].to_numpy()

In [7]:
model_nums = 2
vector_model_num = 0

In [15]:
stats_path = "../../data/AgNews/doc2vec/KMeans/stats/"
stats_list = []
for model_num in range(model_nums):
    stats_list.append(pd.read_csv(f"{stats_path}{model_num}.csv", index_col=0))

In [16]:
stats_list[0]

,mi
2,0.225923
3,0.288683
4,0.395076
6,0.455027
7,0.464722
8,0.456285
9,0.451079
10,0.440101
20,0.267159
40,0.019844


In [17]:
stats_list[1]

,mi
2,0.224843
3,0.288795
4,0.387913
6,0.455016
7,0.464970
8,0.456409
9,0.451782
10,0.440251
20,0.268037
40,0.019861


In [18]:
stats_path = "../../data/AgNews/doc2vec/GMM/stats/"
stats_list_ = []
for model_num in range(model_nums):
    stats_list_.append(pd.read_csv(f"{stats_path}{model_num}.csv", index_col=0))

In [13]:
stats_list[0]

,aic,bic,mi,logl
2,1.251475e+05,1.253705e+05,0.206147,-0.521256
3,3.903846e+05,3.907627e+05,0.265971,-1.626277
4,5.108362e+05,5.114082e+05,0.377064,-2.127993
6,5.796740e+05,5.807502e+05,0.461595,-2.414383
7,5.496084e+05,5.509948e+05,0.477744,-2.288843
8,5.527052e+05,5.544407e+05,0.475559,-2.301447
9,5.111233e+05,5.132466e+05,0.474910,-2.127856
10,4.498888e+05,4.524387e+05,0.466255,-1.872345
20,-7.080704e+05,-6.991217e+05,0.364962,2.957985
40,-4.950850e+06,-4.917469e+06,0.248066,20.657234


In [23]:
pd.concat([stats_list_[0], stats_list[0]], axis=1).drop("logl", axis=1)

,aic,bic,mi,mi
2,1.251475e+05,1.253705e+05,0.206147,0.225923
3,3.903846e+05,3.907627e+05,0.265971,0.288683
4,5.108362e+05,5.114082e+05,0.377064,0.395076
6,5.796740e+05,5.807502e+05,0.461595,0.455027
7,5.496084e+05,5.509948e+05,0.477744,0.464722
8,5.527052e+05,5.544407e+05,0.475559,0.456285
9,5.111233e+05,5.132466e+05,0.474910,0.451079
10,4.498888e+05,4.524387e+05,0.466255,0.440101
20,-7.080704e+05,-6.991217e+05,0.364962,0.267159
40,-4.950850e+06,-4.917469e+06,0.248066,0.019844


In [22]:
pd.concat([stats_list_[1], stats_list[1]], axis=1).drop("logl", axis=1)

,aic,bic,mi,mi
2,1.729385e+05,1.731615e+05,0.224451,0.224843
3,3.904044e+05,3.907826e+05,0.265606,0.288795
4,5.110018e+05,5.115738e+05,0.377571,0.387913
6,5.796896e+05,5.807658e+05,0.461896,0.455016
7,5.496347e+05,5.510211e+05,0.477243,0.464970
8,5.526954e+05,5.544308e+05,0.475696,0.456409
9,5.111163e+05,5.132396e+05,0.474990,0.451782
10,4.497991e+05,4.523490e+05,0.466761,0.440251
20,-7.080268e+05,-6.990781e+05,0.365826,0.268037
40,-4.950913e+06,-4.917532e+06,0.249053,0.019861
